In [1]:
import pandas as pd

#Get the previous date
from datetime import date, timedelta, datetime

#Interacting with Postgres SQL
import psycopg2

import time
from tqdm import tqdm

#importing database
from database_config import postgres as cfg
from report_queries import Reports

In [2]:
#Setting up the connection to the PostgreSQL -feeddate
conn = psycopg2.connect(**cfg)

#this works with the last days of the month as well.
todays_date = str(datetime.today())[:10]

# Queries to Get All active clients

In [3]:
active_client_query = ("""
SELECT
    id,
    name
FROM "public"."customers"
WHERE active = 't';
""")

active_client = pd.read_sql(active_client_query,conn)

In [4]:
rel_client = next(iter(active_client.to_dict().values()))

rel_query = ("""
    SELECT
        DISTINCT ct.customer_id
    FROM "curation_tasks" AS ct
    WHERE started_at >= CURRENT_TIMESTAMP - INTERVAL '1 week'
    AND (ct.resolution IS NULL OR ct.resolution = 'misclassified')""")

rel_client = pd.read_sql(rel_query,conn)
rel_client_weekly = list(rel_client['customer_id'])

active_client = active_client[active_client['id'].isin(rel_client_weekly)]
active_client.set_index('id',drop=True,inplace=True)

customer_dictionary  = next(iter(active_client.to_dict().values()))

# Creating a list to iterate through whenever completing a job. 

In [7]:
customer_ids = list(customer_dictionary.keys())
customer_names = list(customer_dictionary.values())

In [10]:
#creating specific columns to append to the data
agg_data = pd.DataFrame(columns = ['curation_task_id', 'customer_id_x', 'resolution', 'started_at',
       'product_id', 'attribute_name', 'attribute_id', 'Total_NAs',
       'curation_tasks_count', 'customer_id_y', 'bucket_name', 'product_name',
       'active', 'external_id', 'image_url', 'family_friendly'])

agg_data_path = 'C:/Users/groupby/Documents/Python Scripts/NA_Reports/data/Weekly_agg_na.csv'
agg_data.to_csv(agg_data_path,index=False)

for customer in customer_ids:
    temp_na = Reports(conn).client_na_report(customer)
    print(customer_id, 'query and data clean completed')
    temp_na['customer_name'] =  customer_dictionary[customer]
    try:
        temp_products = Reports(conn).client_products(customer)
        print('dropping duplicates')
        temp_products.drop_duplicates(inplace=True)
        print('starting merge')
        agg_data = pd.merge(temp_na,temp_products,on='product_id')
        agg_data.to_csv('C:/Users/groupby/Documents/Python Scripts/NA_Reports/data/Weekly_agg_na.csv',mode='a',index=False,header=False)
        print('on to the next')
    except:
        print("No data available for "+str(customer))

32 query and data clean completed
32 query and data clean completed
100000 Being Processed
100000 Being Processed
100000 Being Processed
100000 Being Processed
100000 Being Processed
100000 Being Processed
100000 Being Processed
100000 Being Processed
100000 Being Processed
100000 Being Processed
100000 Being Processed
100000 Being Processed
100000 Being Processed
100000 Being Processed
100000 Being Processed
100000 Being Processed
100000 Being Processed
100000 Being Processed
100000 Being Processed
100000 Being Processed
100000 Being Processed
100000 Being Processed
100000 Being Processed
100000 Being Processed
100000 Being Processed
100000 Being Processed
100000 Being Processed
100000 Being Processed
100000 Being Processed
100000 Being Processed
100000 Being Processed
100000 Being Processed
100000 Being Processed
100000 Being Processed
100000 Being Processed
100000 Being Processed
100000 Being Processed
100000 Being Processed
100000 Being Processed
100000 Being Processed
100000 Being

In [12]:
if agg_data.shape[0] > 0:
    Gdrive_log_file = "G:/My Drive/Projects/NA_Reports/Weekly_NA_Report/Weekly_NA_Report_log.txt"
    with open(Gdrive_log_file,'r+',encoding='utf-8') as f:
        previous_contents =f.read()
        f.seek(0,0)
        f.write(todays_date+' update has been confirmed.\n')
        f.write(previous_contents)
else:
    with open(Gdrive_log_file,'r+',encoding='utf-8') as f:
        previous_contents =f.read()
        f.seek(0,0)
        f.write(todays_date+' an error has occured and no data has been updated.\n')
        f.write(previous_contents)